In [3]:
from bs4 import BeautifulSoup
from requests import get
import sqlite3
import logging
import json
import string
import random

# Input/ Output
db_path = '../Data/test.db'
db_name = 'test'
log_path = '../Logs/test.log'
conn = sqlite3.connect(db_path)
c = conn.cursor()

# Input
current_file = '../Scraping/already_links_in/techadvisor.txt'
current_reviews = set(line.strip() for line in open(current_file))

logger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

host = "https://www.cnet.com/"
review_cnt = 0
already_cnt = 0
no_annotation = 0
no_rating = 0
no_text = 0
extracted_cnt = 0

def generateNode(length):
    letters_and_digits = string.ascii_letters +  string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

In [52]:
review_cnt = 0
already_cnt = 0
no_annotation = 0
no_rating = 0
no_text = 0
extracted_cnt = 0
# no_anno, ok, ok
my_list =['https://www.cnet.com/reviews/lg-exalt-lte-preview/',
       'https://www.cnet.com/reviews/samsung-galaxy-s20-plus-5g-review/',
       'https://www.cnet.com/reviews/iphone-11-2019-battery-deep-fusion-review/',
        'https://www.cnet.com/products/blu-grand-xl/']
for review_link in my_list:
    review_cnt += 1
    response = get(review_link)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        data = json.loads("".join(soup.find("script", {"type": "application/ld+json"}).contents),strict=False)
        try:
            review = data['review']
            try:
                reviewBody = review['reviewBody']
            except KeyError:
                no_text += 1
                continue
        except KeyError:
            no_annotation += 1
            continue
        node = generateNode(31)
        url = review_link
        try:
            ratingValue = str(review['reviewRating']['ratingValue'])
            bestRating = review['reviewRating']['bestRating']
            worstRating = review['reviewRating']['worstRating']
            reviewRating = "already included"
            print('node:' + node)
            print('url: '+ review_link)
            print('reviewBody: ' + reviewBody)
            print('worstRating: ' + worstRating)
            print('bestRating: ' + bestRating)
            print('ratingValue: ' + ratingValue)
            c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node, url, reviewBody, str(reviewRating), ratingValue, bestRating, worstRating))
            conn.commit()
            extracted_cnt += 1
        except KeyError:
            no_rating += 1
            continue
    except AttributeError:
        no_annotation += 1
        continue

print(f"Done {host} - Reviews extracted: " + str(extracted_cnt) + " out of " + str(review_cnt))
print("Already in: " + str(already_cnt))
print("No Annotations: " + str(no_annotation))
print("No Rating : " + str(no_rating))
print("No ReviewBody: " + str(no_text))

node:_:znodeleqrTvJqNXFbNPaASIEUGVSuED0yeB0
url: https://www.cnet.com/reviews/samsung-galaxy-s20-plus-5g-review/
reviewBody: In Samsung's Galaxy S20 world, the S20 Plus is most in need of an identity. Smack dab in the middle of the smaller, cheaper Galaxy S20 and the gargantuan Galaxy S20 Ultra (with an eye-watering price tag to match), the S20 Plus has very little to differentiate itself from the other two. Pricing starts at $1,199, £999 and AU$1,499.&nbsp;Read more: Motorola Edge Plus is a worthy Samsung Galaxy S20 alternativeThe S20 Plus' two main assets are the 6.7-inch screen (larger than the standard S20's 6.2-inch display), and the fact that it supports the fastest 5G speeds, using mmWave technology. With the exception of Verizon's unreleased model, the Galaxy S20 taps into a slower version of 5G called Sub-6.&nbsp;There's a smattering of other perks. The S20 Plus has a larger battery than the S20 (but smaller than the Ultra), which in our looping video test lasted 21 hours in a

In [4]:
links = []
for page in range(1, 23):
    url = f'https://www.proporta.com/devices?p={page}/'
    response = get(url, headers={'User-Agent': 'Custom'})
    soup = BeautifulSoup(response.text, 'lxml')
    lefts = soup.find_all('li', class_='item first')
    for left in lefts:
        links.append(left.a['href'] + '#allReviews')
    rights = soup.find_all('li', class_='item last')
    for right in rights:
        links.append(right.a['href'] + '#allReviews')
for page in range(1, 5):
    url = f'https://www.proporta.com/accessories?p={page}/'
    response = get(url, headers={'User-Agent': 'Custom'})
    soup = BeautifulSoup(response.text, 'lxml')
    lefts = soup.find_all('li', class_='item first')
    for left in lefts:
        links.append(left.a['href'] + '#allReviews')
    rights = soup.find_all('li', class_='item last')
    for right in rights:
        links.append(right.a['href'] + '#allReviews')

In [31]:
my_list = ['https://www.proporta.com/ted-baker-cheryia-mirror-folio-case-with-outer-card-slot-for-iphone-8-7-6-hedgerow#allReviews']

for review_link in my_list:
    review_cnt += 1
    response = get(review_link, headers={'User-Agent': 'Custom'})
    soup = BeautifulSoup(response.text, 'lxml')
    reviews = soup.find('ol', id='reviewContainer')
    if reviews is None:
        no_annotation += 1
        continue
    else:
        for review in reviews.find_all('li', itemtype='http://schema.org/Review'):
            reviewBody = review.find('p', itemprop='reviewBody').text.split('\r\n')[0]
            ratingValue = review.find('span', itemprop='ratingValue').text
            bestRating = review.find('span', itemprop='bestRating').text
            worstRating = review.find('meta', itemprop='worstRating')['content']
            reviewRating = review.find('div', itemprop='reviewRating').text

            print('node:' + node)
            print('url: '+ review_link)
            print('reviewBody: ' + reviewBody)
            print('worstRating: ' + worstRating)
            print('bestRating: ' + bestRating)
            print('ratingValue: ' + ratingValue)


            # c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node, review_link, reviewBody, reviewRating, ratingValue, bestRating, worstRating))
            # conn.commit()
            # extracted_cnt += 1

node:_:znodePtz2UpHweO6C2XRnHJslWFOEFMPlFtD
url: https://www.proporta.com/ted-baker-cheryia-mirror-folio-case-with-outer-card-slot-for-iphone-8-7-6-hedgerow#allReviews
reviewBody: I use this case for my Iphone 6. I really like the mirror and card slot (had one with a mirror from Ted *** before, but that one didn't have the card holder). Quality is okay.                 
worstRating: 1
bestRating: 5
ratingValue: 5
node:_:znodePtz2UpHweO6C2XRnHJslWFOEFMPlFtD
url: https://www.proporta.com/ted-baker-cheryia-mirror-folio-case-with-outer-card-slot-for-iphone-8-7-6-hedgerow#allReviews
reviewBody: Have had different versions of this case, but this is the only one that doesn't have a magnetic closure, no bother still gorgeous                 
worstRating: 1
bestRating: 5
ratingValue: 4
node:_:znodePtz2UpHweO6C2XRnHJslWFOEFMPlFtD
url: https://www.proporta.com/ted-baker-cheryia-mirror-folio-case-with-outer-card-slot-for-iphone-8-7-6-hedgerow#allReviews
reviewBody: Nice looking case bought as a pr

In [19]:
reviews.find_all('span')


[<span itemprop="author" itemscope="" itemtype="http://schema.org/Person">Review by <span itemprop="name">Natasha V</span></span>,
 <span itemprop="name">Natasha V</span>,
 <span itemprop="ratingValue">5</span>,
 <span itemprop="bestRating">5</span>,
 <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">Review by <span itemprop="name">Rihana Z</span></span>,
 <span itemprop="name">Rihana Z</span>,
 <span itemprop="ratingValue">4</span>,
 <span itemprop="bestRating">5</span>,
 <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">Review by <span itemprop="name">Gary M</span></span>,
 <span itemprop="name">Gary M</span>,
 <span itemprop="ratingValue">5</span>,
 <span itemprop="bestRating">5</span>,
 <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">Review by <span itemprop="name">Brenda S</span></span>,
 <span itemprop="name">Brenda S</span>,
 <span itemprop="ratingValue">5</span>,
 <span itemprop="bestRating">5</span>,
 <span